# Training Notebook

## Setup

### Imports


In [ ]:
import os

from ultralytics import YOLO
from ray import tune


### Tune Model Function


In [ ]:
# Tune model function is broken if yaml is not in root directory

def tune_model(prefix, pt_model, data_dir, data, search_space):

    name = prefix + '_' + pt_model.split('.')[0] + '_' + data.split('.')[0]
    task = pt_model.split('-')[1].split('.')[0]

    if task == 'seg':
        task = 'segment'
    elif task == 'obb':
        task = 'OBB'
    else:
        raise ValueError('Invalid task type. Must be seg or obb')
    
    data_path = os.path.join(data_dir, data)

    model = YOLO(pt_model, task)
    result_grid = model.tune(data=data_path, use_ray=True, space=search_space, grace_period=4, batch=-1, epochs=20, name=name, device=0, degrees=180, gpu_per_trial=1)
    return result_grid


### Train Model Function


In [ ]:
def train_model(prefix, pt_model, data_dir, data, img_size, dropout, seed):

    epochs = 80
    patience = 25

    close_mosaic = 15
    batch_size = -1
    cache = False
    resume = False


    name = prefix + '_' + pt_model.split('.')[0] + '_' + data.split('.')[0]
    task = pt_model.split('-')[1].split('.')[0]

    if task == 'seg':
        task = 'segment'
    elif task == 'obb':
        task = 'obb'
    else:
        raise ValueError('Invalid task type. Must be seg or obb')
    

    data_path = os.path.join(data_dir, data)
    runs_path = os.path.relpath(f"./runs/{task}/{name}")

    # check if folder exists, then check if results.png exists
    # results.png signify end of training, so skip
    # if folder exist, but results.png doesn't, take the model from the folder and continue training
    # if folder doesn't exist, continue as usual

    if os.path.exists(os.path.join(runs_path, "results.png")):
        print(f"Model {name} already trained. Skipping...")
        return
    elif os.path.exists(runs_path):
        pt_model = os.path.join(runs_path, "weights", "last.pt")
        resume = True

    model = YOLO(pt_model)
    model.train(data=data_path, epochs=epochs, imgsz=img_size, patience=patience, batch=batch_size, cache=cache, name=name, dropout=dropout, device=0, degrees=180, close_mosaic=close_mosaic, resume=resume, seed=seed)
    return model


## Training


In [ ]:
SEARCH_SPACE = {
    'lr0': tune.uniform(1e-6, 2e-1),
    'momentum': tune.uniform(0.6, 0.98),
    'box': tune.uniform(0.02, 10),
    'cls': tune.uniform(0.2, 4.0),
    'dropout': tune.uniform(0, 0.5)
}

DATASET_YAML_DIR = os.path.relpath('./datasets_yaml/render_tests')

datasets = []

for file in os.listdir(DATASET_YAML_DIR):
    if file.endswith('.yaml'):
        datasets.append(os.path.join(file))


# model data
# format: 'pretrained model': [image size, dropout]
models_data = {
    'yolov8s-seg.pt': [640, 0.1],
    # 'yolov8s-obb.pt': [800, 0.2],
    # 'yolov9c-seg.pt': [640, 0.1]
}

prefix = 'final'

for i, (pt, v) in enumerate(models_data.items()):
    for data in datasets:
        train_model(prefix, pt, DATASET_YAML_DIR, data, v[0], v[1], 1)
